In [12]:
mport pandas as pd
import numpy as np
import matplotlib as mlp
from sklearn.svm import SVC

In [13]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')    

In [14]:
def preprocess_text(text):
    # Убедитесь, что текст не является NaN или другим нестроковым типом данных
    if pd.isnull(text):
        return set()
    # Токенизация
    tokens = word_tokenize(text.lower())
    # Получение списка стоп-слов
    stop_words = set(stopwords.words('english'))
    # Удаление стоп-слов
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return set(lemmatized_tokens)
# Функция для подсчета количества ключевых действий, присутствующих в предсказанном плане
def count_key_actions(optimal_plan, predicted_plan):
    optimal_actions = preprocess_text(optimal_plan)
    predicted_actions = preprocess_text(predicted_plan)
    # Подсчет количества ключевых действий, присутствующих в обоих планах
    common_actions = optimal_actions.intersection(predicted_actions)
    return len(common_actions), len(optimal_actions)

In [15]:
train_data_path = 'train_data_en.csv'
test_data_path = 'no_labels.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
# Применяем функцию подсчета ключевых действий к каждой строке данных в тренировочном датасете
train_data['optimal_plan_processed'] = train_data['Оптимальный план en'].apply(preprocess_text)
train_data['predicted_plan_processed'] = train_data['Предсказанный план'].apply(preprocess_text)

test_data['optimal_plan_processed'] = test_data['Оптимальный план en'].apply(preprocess_text)
test_data['predicted_plan_processed'] = test_data['Предсказанный план'].apply(preprocess_text)
# Отображаем обновленный тренировочный датасет
train_data.head()


In [16]:
test_data.head()

In [17]:
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from transformers import Trainer, TrainingArguments
# Загрузите предварительно обученный токенизатор и модель BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', num_labels=2) 


In [18]:
import torch

def get_bert_embeddings(text):
    # Токенизируем текст, добавляем специальные токены для BERT и преобразуем в тензоры PyTorch
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Получаем эмбеддинги с помощью модели BERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Возвращает эмбеддинги последнего слоя для первого токена (CLS токена)
    # Это может быть использовано как агрегированное представление всего входного текста
    return outputs.last_hidden_state[:, 0, :].squeeze()


In [19]:
torch.cuda.is_available()

In [20]:
# Предполагается, что train_data - это ваш DataFrame
# Применение функции к столбцу 'text_column_name' и сохранение результатов в новый столбец 'bert_embeddings'
train_data['predicted_plan_embeddings'] = train_data['Предсказанный план'].apply(lambda x: get_bert_embeddings(x).numpy())
train_data['optimal_plan_embeddings'] = train_data['Оптимальный план en'].apply(lambda x: get_bert_embeddings(x).numpy())
# После выполнения этого шага каждая строка в 'bert_embeddings' будет содержать вектор эмбеддингов для соответствующего текста
train_data.head()

In [21]:
test_data['predicted_plan_embeddings'] = test_data['Предсказанный план'].apply(lambda x: get_bert_embeddings(x).numpy())
test_data['optimal_plan_embeddings'] = test_data['Оптимальный план en'].apply(lambda x: get_bert_embeddings(x).numpy())
# После выполнения этого шага каждая строка в 'bert_embeddings' будет содержать вектор эмбеддингов для соответствующего текста
test_data.head()

In [22]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Предполагается, что train_data уже содержит столбцы с эмбеддингами
# Преобразуем эмбеддинги из списка в отдельные признаки
def embeddings_to_features(data, column_prefix):
    embeddings = np.stack(data[column_prefix + '_embeddings'].values)
    feature_names = [f"{column_prefix}_embedding_{i}" for i in range(embeddings.shape[1])]
    features_df = pd.DataFrame(embeddings, columns=feature_names, index=data.index)
    return features_df

# Преобразование эмбеддингов в признаки
optimal_plan_features = embeddings_to_features(train_data, 'optimal_plan')
predicted_plan_features = embeddings_to_features(train_data, 'predicted_plan')

# Объединяем все признаки в один DataFrame
features = pd.concat([optimal_plan_features, predicted_plan_features], axis=1)

# Предположим, что целевая переменная называется 'success'
labels = train_data['Успех предсказанного плана']

# Разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [23]:
# Создаем и обучаем модель
model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.05, num_leaves=31, objective='binary')
model.fit(X_train, y_train)

# Делаем предсказания на тестовом наборе
predictions = model.predict(X_test)

# Вычисляем точность
accuracy = accuracy_score(y_test, predictions)
print(f"Точность модели: {accuracy:.2f}")


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

t_data = train_data
ts_data = test_data
# Для train_data
X_train_data = [np.concatenate([optimal, predicted]) for optimal, predicted in zip(t_data['optimal_plan_embeddings'].values, t_data['predicted_plan_embeddings'].values)]

y_train_data = t_data['Успех предсказанного плана'].values

# Для test_data
X_test_data = [np.concatenate([optimal, predicted]) for optimal, predicted in zip(ts_data['optimal_plan_embeddings'].values, ts_data['predicted_plan_embeddings'].values)]
# y_test_data = t_data['Успех предсказанного плана'].values

# Теперь X_train_data и X_test_data содержат конкатенированные векторы признаков
# Используйте X_train_data и y_train_data для разделения на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_train_data, y_train_data, test_size=0.2, random_state=42)

# После этого, вы можете приступить к обучению модели, используя X_train и y_train
# А X_test_data и y_test_data использовать для тестирования модели


In [33]:
 class VectorDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class TestDataset(Dataset):
    def __init__(self, features):
        self.features = torch.tensor(features, dtype=torch.float32)

    def __len__(self):
        return len(self.features)
  
    def __getitem__(self, idx):
        return self.features[idx]

train_dataset = VectorDataset(X_train, y_train)
test_dataset = TestDataset(X_test_data)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(type(test_dataset))
print(type(train_dataset))

In [34]:
X_train = np.array(X_train)
X_val = np.array(X_val)
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Определим размер входа как размерность одного вектора эмбеддинга
input_size = X_train.shape[1]
num_classes = len(set(y_train))

model = SimpleNN(input_size, num_classes)
print(model)
print("Input size:", input_size, "Number of classes:", num_classes)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.train()
# Обучение модели
num_epochs = 100
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')




In [37]:
# Переключение модели в режим оценки для тестирования
model.eval()
test_predictions = []

with torch.no_grad():  # Отключение вычисления градиентов для ускорения
    for features in test_loader:
        features = features.to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        test_predictions.extend(predicted.cpu().numpy())

# Вывод первых 10 предсказаний
print("Test predictions for the first 10 samples:", test_predictions[:])
# Создание DataFrame из предсказаний
predictions_df = pd.DataFrame({
    'Test Predictions': test_predictions
})

# Сохранение DataFrame в CSV
predictions_df.to_csv('test_predictions.csv', index=False)